In [253]:
import numpy as np
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV,RandomizedSearchCV
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE,RandomOverSampler
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
import dtale

In [254]:
df=pd.read_csv(r'C:\Users\Pradeep Sharma\Downloads\Case_Study_Data_Train.csv')
dtale.show(df)

In [255]:
df

,Package,Sample,Medical_Description
0,Neurosurgery,VP Shunt Placement,"TITLE OF OPERATION: , Placement of right new v..."
1,Neurosurgery,Craniotomy - Frontal Zygomatic,"PREOPERATIVE DIAGNOSIS:, Metopic synostosis w..."
2,Neurosurgery,Wound Closure & Debridement - Hydrocephalus,"TITLE OF OPERATION:, A complex closure and de..."
3,Urology,Foul-Smelling Urine,"CHIEF COMPLAINT:, Foul-smelling urine and sto..."
4,Neurology,MRI Head,"EXAM:, MRI Head W&WO Contrast.,REASON FOR EXA..."
...,...,...,...
1807,Neurology,MRI Brain - Toxoplasmosis,"CC:, Lethargy.,HX:, This 28y/o RHM was admitte..."
1808,Gastroenterology,Esophagogastroduodenoscopy - 13,"PROCEDURE PERFORMED: , Esophagogastroduodenosc..."
1809,Ophthalmology,Optometry - Letter,"RE: Sample Patient,Dear Dr. Sample:,Sample Pa..."
1810,Hematology - Oncology,Parathyroid Adenoma Excision,"PREOPERATIVE DIAGNOSIS:, Right superior parat..."


In [256]:
df.columns

Index(['Package', 'Sample', 'Medical_Description'], dtype='object')

In [257]:
df.isnull().sum()

Package                 0
Sample                  0
Medical_Description    15
dtype: int64

In [258]:
df.dropna(axis=0,inplace=True)
df.reset_index(drop=True,inplace=True)

In [259]:
df.shape

(1797, 3)

In [260]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
df['Sample_label_encoding']=label_encoder.fit_transform(df['Sample'])
df['Package_label_encoding']=label_encoder.fit_transform(df['Package'])

In [261]:
def preprocess(tweet):
   
   # Removing handles and hashtags
   tweet = re.sub('@[^\s]+','',str(tweet))
   tweet = re.sub(r'#', '', str(tweet))
   # Removing URLS
   tweet= re.sub('((www\.[^\s]+)|(https?://[^\s]+))','',str(tweet))
   #removing all punchuation and special character
   tweet = re.sub('[^a-zA-Z]',' ',str(tweet))
   #removing extra white space
   tweet = re.sub('[\s]+', ' ', str(tweet))
   tweet = re.sub('[\n]+', ' ', str(tweet))
   tweet=tweet.lower()
   return tweet
train_data= df['Medical_Description'].apply(lambda x:preprocess(x))
type(train_data)

pandas.core.series.Series

In [262]:

train_data=pd.DataFrame(train_data)
train_data.columns=['Medical_Description']

In [263]:
train_data.reset_index(drop=True,inplace=True)

In [264]:
train_data.columns

Index(['Medical_Description'], dtype='object')

In [265]:

def token(tweet):
  tweet = word_tokenize(tweet)
  return tweet
train_data['Medical_Description']=train_data['Medical_Description'].apply(lambda x:token(x))

In [266]:
train_data

,Medical_Description
0,"[title, of, operation, placement, of, right, n..."
1,"[preoperative, diagnosis, metopic, synostosis,..."
2,"[title, of, operation, a, complex, closure, an..."
3,"[chief, complaint, foul, smelling, urine, and,..."
4,"[exam, mri, head, w, wo, contrast, reason, for..."
...,...
1792,"[cc, lethargy, hx, this, y, o, rhm, was, admit..."
1793,"[procedure, performed, esophagogastroduodenosc..."
1794,"[re, sample, patient, dear, dr, sample, sample..."
1795,"[preoperative, diagnosis, right, superior, par..."


In [267]:
#Performing Text Normalization
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
lem = WordNetLemmatizer()
def normalize(tweet):
  tweet = [lem.lemmatize(word) for word in tweet if not word in stop_words  ]
  #print(tweet)
  tweet = ' '.join(tweet)
  return tweet
train_data['Medical_Description']= train_data['Medical_Description'].apply(lambda x :normalize(x))
train_data.dropna(inplace=True)

In [268]:
train_data

,Medical_Description
0,title operation placement right new ventriculo...
1,preoperative diagnosis metopic synostosis trig...
2,title operation complex closure debridement wo...
3,chief complaint foul smelling urine stomach pa...
4,exam mri head w wo contrast reason exam dyspne...
...,...
1792,cc lethargy hx rhm admitted local hospital mar...
1793,procedure performed esophagogastroduodenoscopy...
1794,sample patient dear dr sample sample patient s...
1795,preoperative diagnosis right superior parathyr...


In [270]:
x_tf=train_data['Medical_Description']
y_tf=df['Package_label_encoding']

In [271]:
x_tf

0       title operation placement right new ventriculo...
1       preoperative diagnosis metopic synostosis trig...
2       title operation complex closure debridement wo...
3       chief complaint foul smelling urine stomach pa...
4       exam mri head w wo contrast reason exam dyspne...
                              ...                        
1792    cc lethargy hx rhm admitted local hospital mar...
1793    procedure performed esophagogastroduodenoscopy...
1794    sample patient dear dr sample sample patient s...
1795    preoperative diagnosis right superior parathyr...
1796    chief complaint rule obstructive sleep apnea s...
Name: Medical_Description, Length: 1797, dtype: object

In [272]:
from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format(r'D:\nlp\GoogleNews-vectors-negative300.bin\GoogleNews-vectors-negative300.bin', binary=True)

2021-06-15 16:07:10,982 - INFO     - loading projection weights from D:\nlp\GoogleNews-vectors-negative300.bin\GoogleNews-vectors-negative300.bin
2021-06-15 16:07:28,241 - INFO     - loaded (3000000, 300) matrix from D:\nlp\GoogleNews-vectors-negative300.bin\GoogleNews-vectors-negative300.bin


In [273]:
tfidf = TfidfVectorizer()
tfidf.fit(x_tf)
dictionary = dict(zip(tfidf.get_feature_names(), list(tfidf.idf_)))

In [274]:
def word2vectransform(list_of_sentance):
    zero = np.zeros(300)
    tfidf_sent_vectors = []; # the tfidf-w2v for each sentence/review is stored in this list
    for sent in list_of_sentance: # for each review/sentence 
        sent = sent.split()
        sent_vec = np.zeros(300) # as word vectors are of zero length
        weight_sum =0; # num of words with a valid vector in the sentence/review
        for word in sent: # for each word in a review/sentence
            try:
                vec = model[word]
            except:
                vec = zero
            try:
                tf_idf = dictionary[word]*(sent.count(word)/len(sent))
            except:
                tf_idf = 0
            sent_vec += (vec * tf_idf)
            weight_sum += tf_idf
        if weight_sum != 0:
                sent_vec /= weight_sum
        tfidf_sent_vectors.append(sent_vec)
        
    return tfidf_sent_vectors

In [275]:
df1 = word2vectransform(x_tf)

In [276]:
len(df1)


1797

In [277]:
df1=pd.DataFrame(df1)
df1

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-0.057511,0.082052,0.001945,-0.002946,-0.161117,-0.017205,0.017806,-0.084625,0.045700,-0.022220,...,-0.040396,0.025264,-0.109417,0.036507,-0.057018,0.031657,0.014667,-0.006849,0.069431,0.014730
1,-0.127871,0.032782,-0.065683,0.045283,-0.205079,-0.027810,0.112507,-0.222881,-0.027920,0.016951,...,-0.032504,0.138641,-0.057223,0.064797,0.007579,0.031907,-0.029291,-0.001082,-0.022982,0.029634
2,0.013128,0.138364,0.035924,-0.015228,-0.184354,0.017934,-0.028812,-0.030057,0.063415,-0.044309,...,-0.034458,0.046147,-0.100267,0.037911,-0.025378,0.056931,-0.044435,-0.009380,0.087705,0.032640
3,-0.033980,0.034267,-0.018043,0.093681,-0.128063,0.072707,0.043473,-0.096468,0.039397,0.048985,...,-0.032277,-0.007698,0.002436,0.049742,-0.015009,0.100311,-0.120365,0.067838,0.101571,0.056460
4,-0.021907,0.053769,0.006281,0.019758,-0.126262,0.027823,0.013556,-0.132247,0.057740,0.060282,...,-0.043282,0.026800,0.027557,-0.030782,-0.035726,-0.008092,-0.052682,-0.046732,0.115855,0.041834
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1792,-0.024788,0.069206,0.001767,0.071475,-0.096801,0.023937,0.033385,-0.108177,0.041325,0.008989,...,-0.026201,-0.003089,-0.018412,0.037784,0.002808,0.050242,-0.046127,0.021686,0.064246,0.054669
1793,-0.018717,0.088762,0.001588,0.012388,-0.162173,0.041680,0.048625,-0.064608,0.091669,0.041772,...,-0.055383,-0.038093,-0.010295,0.006173,-0.047703,0.106725,-0.030956,0.045059,0.064720,-0.004670
1794,-0.012166,0.029125,-0.052104,0.089817,-0.050441,0.086020,0.040196,-0.048264,0.054938,0.058255,...,-0.085619,-0.003370,-0.051692,-0.015924,0.008525,-0.024500,0.004636,0.031259,0.017080,-0.052205
1795,-0.079749,0.118938,0.052479,0.011187,-0.193901,0.022671,0.042062,-0.097781,0.061835,0.010108,...,0.008344,0.026661,-0.001007,0.025665,0.012566,0.038637,-0.111465,-0.005363,0.105542,0.046521


In [278]:
df_final=pd.concat([df1,df['Sample_label_encoding']],axis=1)

In [279]:
df_final.shape

(1797, 301)

In [290]:
x = df_final.iloc[:,0:301]
y=pd.DataFrame(y_tf)
y.columns=['Package']


In [291]:
x

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,Sample_label_encoding
0,-0.057511,0.082052,0.001945,-0.002946,-0.161117,-0.017205,0.017806,-0.084625,0.045700,-0.022220,...,0.025264,-0.109417,0.036507,-0.057018,0.031657,0.014667,-0.006849,0.069431,0.014730,1577
1,-0.127871,0.032782,-0.065683,0.045283,-0.205079,-0.027810,0.112507,-0.222881,-0.027920,0.016951,...,0.138641,-0.057223,0.064797,0.007579,0.031907,-0.029291,-0.001082,-0.022982,0.029634,469
2,0.013128,0.138364,0.035924,-0.015228,-0.184354,0.017934,-0.028812,-0.030057,0.063415,-0.044309,...,0.046147,-0.100267,0.037911,-0.025378,0.056931,-0.044435,-0.009380,0.087705,0.032640,1616
3,-0.033980,0.034267,-0.018043,0.093681,-0.128063,0.072707,0.043473,-0.096468,0.039397,0.048985,...,-0.007698,0.002436,0.049742,-0.015009,0.100311,-0.120365,0.067838,0.101571,0.056460,705
4,-0.021907,0.053769,0.006281,0.019758,-0.126262,0.027823,0.013556,-0.132247,0.057740,0.060282,...,0.026800,0.027557,-0.030782,-0.035726,-0.008092,-0.052682,-0.046732,0.115855,0.041834,1013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1792,-0.024788,0.069206,0.001767,0.071475,-0.096801,0.023937,0.033385,-0.108177,0.041325,0.008989,...,-0.003089,-0.018412,0.037784,0.002808,0.050242,-0.046127,0.021686,0.064246,0.054669,996
1793,-0.018717,0.088762,0.001588,0.012388,-0.162173,0.041680,0.048625,-0.064608,0.091669,0.041772,...,-0.038093,-0.010295,0.006173,-0.047703,0.106725,-0.030956,0.045059,0.064720,-0.004670,640
1794,-0.012166,0.029125,-0.052104,0.089817,-0.050441,0.086020,0.040196,-0.048264,0.054938,0.058255,...,-0.003370,-0.051692,-0.015924,0.008525,-0.024500,0.004636,0.031259,0.017080,-0.052205,1171
1795,-0.079749,0.118938,0.052479,0.011187,-0.193901,0.022671,0.042062,-0.097781,0.061835,0.010108,...,0.026661,-0.001007,0.025665,0.012566,0.038637,-0.111465,-0.005363,0.105542,0.046521,1214


In [292]:
y

,Package
0,6
1,6
2,6
3,11
4,5
...,...
1792,5
1793,2
1794,8
1795,3


In [293]:
train_x,test_x,train_y,test_y = train_test_split(x,y,test_size=.2,random_state=1,stratify=y)

In [294]:
oversample = RandomOverSampler(random_state=12)
X_over, y_over = oversample.fit_resample(train_x, train_y)

In [295]:
y_over.value_counts()

Package
0          267
1          267
2          267
3          267
4          267
5          267
6          267
7          267
8          267
9          267
10         267
11         267
dtype: int64

In [296]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier,OneVsOneClassifier
model = LogisticRegression(random_state=0,C=0.5,solver='newton-cg',class_weight='balanced',multi_class='ovr',n_jobs=-1)
ovr = OneVsRestClassifier(model)
ovr.fit(X_over,y_over) 
print(ovr.score(X_over,y_over))
print(ovr.score(train_x,train_y))
print(ovr.score(test_x,test_y))

0.83458177278402
0.8037578288100209
0.7555555555555555


In [297]:
zz=ovr.predict(test_x)
print(classification_report(test_y,zz))

              precision    recall  f1-score   support

           0       0.90      0.82      0.86        67
           1       0.59      0.59      0.59        17
           2       0.89      0.78      0.83        40
           3       0.55      0.69      0.61        16
           4       0.60      0.60      0.60        15
           5       0.70      0.75      0.72        40
           6       0.47      0.82      0.60        17
           7       0.92      0.82      0.87        28
           8       0.83      1.00      0.91        15
           9       0.84      0.73      0.78        64
          10       0.31      0.38      0.34        13
          11       0.92      0.79      0.85        28

    accuracy                           0.76       360
   macro avg       0.71      0.73      0.71       360
weighted avg       0.78      0.76      0.76       360



In [298]:
from sklearn.metrics import confusion_matrix
pd.DataFrame(confusion_matrix(test_y, zz))

,0,1,2,3,4,5,6,7,8,9,10,11
0,55,3,2,2,0,1,1,1,0,1,1,0
1,0,10,0,0,0,1,1,0,1,1,3,0
2,1,0,31,2,4,0,0,1,0,1,0,0
3,0,0,0,11,0,2,0,0,1,1,1,0
4,2,0,2,2,9,0,0,0,0,0,0,0
5,2,0,0,0,0,30,7,0,0,1,0,0
6,0,0,0,0,0,0,14,0,0,2,1,0
7,0,0,0,1,0,0,0,23,1,0,2,1
8,0,0,0,0,0,0,0,0,15,0,0,0
9,0,1,0,0,0,7,6,0,0,47,3,0


In [299]:
import joblib
joblib.dump(ovr, 'logistic_med_with_out_sample_col.pkl')

['logistic_med_with_out_sample_col.pkl']